<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/acme.jpeg width=100 align="left">

# Tokenize Data

There are organizations that care more about the irreversibility of the tokenized data and not so much about preserving the original data format.

Tokenization replaces sensitive data with unique values (tokens) that are unrelated to the origina value.  Those tokens cannot risk exposing the plaintext satifing the PCI-DSS guidance.

<img src="images/vault-tokenization-1.png">

**Characteristics of tokenization:** 

* non-reversible
* integrated metadata:  supports metdata for identifying data type and purpose
* extreme scale and performance 


**Prerequisites:**
* Vault Enterprise binary with Advanced Data Protection
* Vault License

## Setup

In [ ]:
/bin/rm -rf $HOME/demos/vault_cluster/vrd/data/*

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [ ]:
which vault
vault --version
vault login root

\
**EXECUTE the following command in SEPARATE window**

```
VAULT_UI=true VAULT_REDIRECT_ADDR=http://127.0.0.1:8200 evault server -log-level=trace -dev -dev-root-token-id=root -dev-listen-address=127.0.0.1:8200 -dev-ha -dev-transactional
```
OR   
`evrd1`   

## (Optional)  Apply the license

In [ ]:
curl \
    --insecure \
    --header "X-Vault-Token: $VAULT_ROOT_TOKEN" \
    --request PUT \
    --data @/Users/tio/Documents/vault_license.json \
    $VAULT_ADDR/v1/sys/license

In [ ]:
vault status

##### Notice Initialized (=true) and Sealed (=false) status 

## Enable the Transform secrets engine

In [ ]:
vault secrets enable transform

## Roles, Transformations, Templates and Alphabets

Create a role `mobile-pay` which is attached to `credit-card` transformation.  The tokenized value has a fixed maximum time-to-live of 24 hours.  

<img src=images/vault-tokenization-2.png >


### Create a Role

In [ ]:
vault write transform/role/mobile-pay transformations=credit-card

### Specify the transformation

In [ ]:
vault write transform/transformation/credit-card \
  type=tokenization \
  max_ttl=24h \
  allowed_roles=mobile-pay

### Transform it
After the secrets engine is configured and a user/machine has a Vault token, it can use this secret engine to encode and decode input values

In [ ]:
vault write transform/encode/mobile-pay value=1111-2222-3333-4444 \
  ttl=8h \
  metadata="type=Amex" \
  metadata="organization=HashiCorp"

### Decode some input value using the /decode endpoint with a named role:

In [ ]:
vault write transform/decode/mobile-pay value=$MY_ENCODED_CCN

In [ ]:
export MY_ENCODED_CCN="9UrVvkyFti3nvRDa5xN2kkDGuudWAtWTpfpNKi3ezVfnY94EtDqAPPeLLeeerKEJMDp7iece"
vault write transform/metadata/mobile-pay value=$MY_ENCODED_CCN

In [ ]:
vault write transform/validate/mobile-pay value=$MY_ENCODED_CCN
echo $MY_ENCODED_CCN

In [ ]:
vault write transform/tokenized/mobile-pay value=$MY_ENCODED_CCN

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">